In [ ]:
#Ensuring compliance of code with both python2 and python3
#!pip install ipyparallel
#!pip install jpype1
#!pip install SALib
#!pip install pyNetLogo
from __future__ import division, print_function
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

#Import the sampling and analysis modules for a Sobol variance-based 
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
problem = { 
  'num_vars': 6,
  'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}
n = 1000
param_values = saltelli.sample(problem, n, calc_second_order=True)
param_values.shape


(14000L, 6L)

In [ ]:
!pip uninstall nl4py
!pip install nl4py
import nl4py
nl4py.startServer()

import time
startTime = int(round(time.time() * 1000))
nl4py.deleteAllHeadlessWorkspaces() 

runsDone = 0
runsStarted = 0
runsNeeded = 1000
allResults = []
workspace = nl4py.newNetLogoHeadlessWorkspace()
workspace.openModel('Wolf Sheep Predation.nlogo')
print(workspace.getParamNames())
def openAndSchedule(workspace_):
    workspace_.openModel('Wolf Sheep Predation.nlogo')
    workspace_.command("stop")
    '''for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            workspace_.command('random-seed {}'.format(param_values[runsStarted][i]))
        else:
            #Otherwise, assume the input parameters are global variables
            workspace_.command('set {0} {1}'.format(name, param_values[runsStarted][i]))'''
    workspace_.setParamsRandom()
    workspace_.command('setup')
    workspace_.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")    

for i in range(0,200):
    workspace = nl4py.newNetLogoHeadlessWorkspace()
    openAndSchedule(workspace)
    runsStarted = runsStarted + 1
while (runsDone < runsNeeded):
    print("yo")
    for workspace in nl4py.getAllHeadlessWorkspaces():
        print("a")
        allResults.extend(workspace.getScheduledReporterResults())
        print("b")
        runsDone = runsDone + 1
        print(runsDone)
        if runsStarted < runsNeeded:
            openAndSchedule(workspace)
            runsStarted = runsStarted + 1
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

In [2]:
#!pip install nl4py
import os
import nl4py
import ipyparallel

client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))
#Push the "problem" variable from the notebook to a corresponding variable on the engines
direct_view.push(dict(problem=problem))
nl4py.startServer()

Dependencies installed successfully! 
Start the NetLogoControllerServer using nl4py.startServer()
Shutting down old server instance...
[0, 1, 2, 3, 4, 5, 6, 7]
Server started.


In [6]:
%%px 

import os
os.chdir(cwd)
import time
import nl4py
nl4py.startServer()
import pandas as pd
netlogo = workspace = nl4py.newNetLogoHeadlessWorkspace()
netlogo.openModel('Wolf Sheep Predation.nlogo')

[stdout:0] Server started.
[stdout:1] Server started.
[stdout:2] Server started.
[stdout:3] Server started.
[stdout:4] Server started.
[stdout:5] Server started.
[stdout:6] Server started.
[stdout:7] Server started.


In [7]:
def simulation(experiment):
    
    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")       
    
    results = netlogo.getScheduledReporterResults()
    
    return results

In [ ]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

In [13]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

2111767


In [14]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

878631


In [15]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

1316986


In [16]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

854811


In [17]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

1323916


In [19]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

885860


In [21]:
import time
startTime = int(round(time.time() * 1000))
lview = client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(simulation, param_values[0:1000]))
stopTime = int(round(time.time() * 1000))
print(stopTime - startTime)

887937


In [ ]:
results.to_csv('Sobol_parallel.csv')

In [ ]:
results.head(5)


In [5]:
import nl4py
nl4py.startServer()
n = nl4py.netlogoWorkspaceFactory.newNetLogoHeadlessWorkspace()
n.openModel("./Wolf Sheep Predation.nlogo")
n.setParamsRandom()
n.command("setup")
n.scheduleReportersAndRun(["ticks","count sheep", "count wolves"],0,1,100,"go")
print(n.getScheduledReporterResults())

Server started.


AttributeError: 'NoneType' object has no attribute 'getParamSpecs'